Apuração da Premiação Tina (Individual)



In [294]:
import pandas as pd
import numpy as np
from pathlib import Path

In [295]:
pd.set_option('display.max_columns', None)

In [296]:
# Dia da apuração
# TODO TROCAR A DATA PARA O ANALISTA DE NEGÓCIOS RODAR NO DIA DA APURAÇÃO
# hoje = pd.to_datetime('today').date()
hoje = pd.to_datetime('2022-05-27')
ANO=str(hoje.year)

In [297]:
# TODO FAZER LOOP PARA RODAR APENAS UMA VEZ 
# SELECIONANDO EMPRESA
ID_UNN=1

dict_EMPRESAS={1:"TELHANORTE", 4:"TUMELERO"}
empresa=dict_EMPRESAS[ID_UNN]
print(ID_UNN)
print(empresa)

1
TELHANORTE


In [298]:
# DIA PADRÃO DE APURAÇÃO
#DIA_APURA=27
DIA_APURA=hoje.day

In [299]:
DIA_APURA

27

In [300]:
MESES= ['jan','fev','mar','abr','mai','jun','jul','ago','set','out','nov','dez']
I_MESES= ['01','02','03','04','05','06','07','08','09','10','11','12']
MES=MESES[hoje.month-1]
I_MES=I_MESES[MESES.index(MES)]
I_MES_ANTERIOR=I_MESES[MESES.index(MES)-1]


In [301]:
I_MES_ANTERIOR

'04'

In [302]:
# Período de Apuração   -   Faltas injustificadas no período
# 27 de Janeiro de 2022
DATA_I = pd.to_datetime('2022-'+I_MES_ANTERIOR+'-'+str(DIA_APURA))  # data inicial AAAA-MM-DD
DATA_F = pd.to_datetime('2022-'+I_MES+'-'+str(DIA_APURA-1))  # data final AAAA-MM-DD

In [303]:
print(DATA_I)
print(DATA_F)

2022-04-27 00:00:00
2022-05-26 00:00:00


In [304]:
# PASTAS 

DIR_BASE='C:/apuracao/'
DIR_INFO=DIR_BASE+'info/'
DIR_TINA_BASES=DIR_INFO
DIR_ENTRA=DIR_BASE+'ENTRADA/'
# PASTA PARA MANTER AS BASES ORIGINAIS
DIR_IN=DIR_BASE+ANO+'/'+MES+'/ORIGINAIS/'
# PASTA COM PLANILHA DE CHECAGEM PARA O ANALISTA DE NEGÓCIOS
DIRETORIO_CHECAGEM=DIR_BASE+ANO+'/'+MES+'/CHECAGEM/'
# PASTA PARA PLANILHAS DAS VERBAS PARA RH 
DIR_RH=DIR_BASE+ANO+"/"+MES+"/"+"para_RH/"



In [305]:
# CRIANDO PASTAS DE SAÍDA SE NÃO EXISTIREM
Path(DIR_IN).mkdir(parents=True, exist_ok=True)
Path(DIRETORIO_CHECAGEM).mkdir(parents=True, exist_ok=True)
Path(DIR_RH).mkdir(parents=True, exist_ok=True)


In [306]:
# Arquivos a serem importados
# Incluídas lojas TUMELERO
F_ALVO = DIR_INFO+"target_salesperson.csv" # 
F_VERBAS = DIR_INFO+"/verbas_descricao.xlsx"

F_LOJA_SAFRA=DIR_INFO+"lojas.csv" # 
#f_loja_safra = "info/lojas_pilotos_safra3.xlsx" # Exemplo de janeiro


In [307]:
nome_lojas = pd.read_csv(F_LOJA_SAFRA, encoding="ISO-8859-1", delimiter=";")

In [308]:
# TODO Colocar os valores dos parâmetros em arquivo e importar 
# Parâmetros do Modelo
# Modelo Vendedores
# Modelo Vendedores Comissão
gat_min_comissao = 0.0  # Gatilho minimo comissão
ANG_COEF_COMISSAO = 0.01  # Coeficiente angular = 1%
lin_coef_comissao = 0.0  # Coeficiente Linear = 0.0%

# Modelo Vendedores Comissão Turbinada Decoração
# Como as safras 1, 2 e 3 tem os mesmos valores, mantive só um valor para todas as safras
gat_min_decoracao = 0.0  # Gatilho minimo decoração
ANG_COEF_DECORACAO = 0.02  # Coeficiente angular decoração = 2%
lin_coef_decoracao = 0.0  # Coeficiente Linear decoração = 0.0%

# Modelo Vendedores Comissão turbinada Áreas Técnicas
# Como as safras 1, 2 e 3 tem os mesmos valores, mantive só um valor para todas as safras
gat_min_areastecnicas = 0.0  # Gatilho minimo Áreas Técnicas = 0.0%
ANG_COEF_AREASTECNICAS= 0.005  # Coeficiente angular Áreas Técnicas = 0.5%
lin_coef_areastecnicas = 0.0  # Coeficiente Linear decoração = 0.0%

# Modelo Vendedores Prêmio superação vendas
M1 = 100  # meta1 = 100%
M2 = 150  # meta2 = 150%
M3 = 200  # meta3 = 200%
M4 = 250  # meta4 = 250%
B1 = {1:300, 4:200}  # bonus1 -> ganha se bater a meta1; Telhanorte 1 = R$300,00; Tumelero 4= R$200,00
B2 = {1:600, 4:400}  # bonus2 -> ganha se bater a meta2; Telhanorte 1 = R$600,00; Tumelero 4= R$400,00
B3 = {1:900, 4:600}  # bonus3 -> ganha se bater a meta3; Telhanorte 1 = R$900,00; Tumelero 4= R$600,00
B4 = {1:1200, 4:800}  # bonus4  -> ganha se bater a meta4; Telhanorte 1 = R$1200,00; Tumelero 4= R$800,00

# Modelo Vendedores Prêmio Rentabilidade
GAT_MIN_RENTAB = 100.  # Tem que bater 100% da rentabilidade
ANG_COEF_RENTAB = 0.001  # Coeficiente angular da rentabilidade = 0.1%
LIN_COEF_RENTAB = -0.1  # Coeficiente linear da rentabilidade = -0.1
KICKER = {1:200., 4:100} # Premiação inicial quando a rentabilidade alcança 100%; 
                                   # Telhanorte 1 = R$200,00; Tumelero 4 = R$100,00

In [309]:
# Parâmetros do Modelo
# Modelo Assessores
ASS_A1 = 30000  # Alvo1 = R$30.000,00
ASS_A2 = 50000  # Alvo2 = R$50.000,00
ASS_B1 = 100  # Bônus1 caso bata Alvo1
ASS_B2 = 200  # Bônus2 caso bata Alvo2


In [310]:
# Vitor (Wingg) via e-mail no dia da apuração
E_EXTRATO = DIR_ENTRA+"extract_remuneration_"+ANO+I_MES+str(DIA_APURA)+".xlsx"
F_EXTRATO = DIR_IN+"extract_remuneration_"+ANO+I_MES+str(DIA_APURA)+".xlsx"

# Thayna via sharepoint antes da apuração
# F_FALTAS = DIR_IN+"FALTAS_INJUSTIFICADAS_PREMIACOES TINA_23mar2022_ThaynaCarneiro.xlsx"
E_FALTAS = DIR_ENTRA+"PREMIACOES TINA.xlsx"
F_FALTAS = DIR_IN+"PREMIACOES TINA.xlsx"

In [311]:
print('A planilha de checagem da Apuração da PREMIAÇÃO TINA INDIVIDUAL realizada em '+MES+' está aqui:')
print(DIRETORIO_CHECAGEM)
# ARQUIO DE SAÍDA PARA CHECAGEM DO ANALISTA DE NEGÓCIOS - EM EXCEL
F_SAI_CHECAGEM=DIRETORIO_CHECAGEM + "checagem_apuração_PREMIACAO_INDIVIDUAL_"+MES+".xlsx"
print(F_SAI_CHECAGEM)


A planilha de checagem da Apuração da PREMIAÇÃO TINA INDIVIDUAL realizada em mai está aqui:
C:/apuracao/2022/mai/CHECAGEM/
C:/apuracao/2022/mai/CHECAGEM/checagem_apuração_PREMIACAO_INDIVIDUAL_mai.xlsx


In [312]:
# Sufixos possíveis
sufixo="_sem_considerar_faltas_injustificadas"
#sufixo="_analista"

In [313]:
def nome_verbas(DIR_RH, empresa):
    # Verbas para o RH das premiações Tina Individuais
    F_VERBA_ASS=DIR_RH+"VERBA_4118_SEGMENTO_ASSESSORES_"+empresa+".xlsx"
    F_VERBA_VEN_REN=DIR_RH+"VERBA_4119_RENTABILIDADE_VENDAS_"+empresa+".xlsx"
    F_VERBA_VEN_DEC=DIR_RH+"VERBA_4120_DECORACAO_VENDAS_"+empresa+".xlsx"
    F_VERBA_VEN_SUP=DIR_RH+"VERBA_4121_SUPERACAO_VENDAS_"+empresa+".xlsx"
    F_VERBA_VEN_TEC=DIR_RH+"VERBA_4122_AREAS_TECNICAS_VENDAS_"+empresa+".xlsx"
    # Verbas para o RH das premiações Tina Individuais SEM CONSIDERAR FALTAS INJUSTIFICADAS
    F_VERBA_ASS_SEM=DIR_RH+"VERBA_4118_SEGMENTO_ASSESSORES_"+empresa+sufixo+".xlsx"
    F_VERBA_VEN_REN_SEM=DIR_RH+"VERBA_4119_RENTABILIDADE_VENDAS_"+empresa+sufixo+".xlsx"
    F_VERBA_VEN_DEC_SEM=DIR_RH+"VERBA_4120_DECORACAO_VENDAS_"+empresa+sufixo+".xlsx"
    F_VERBA_VEN_SUP_SEM=DIR_RH+"VERBA_4121_SUPERACAO_VENDAS_"+empresa+sufixo+".xlsx"
    F_VERBA_VEN_TEC_SEM=DIR_RH+"VERBA_4122_AREAS_TECNICAS_VENDAS_"+empresa+sufixo+".xlsx"

    return {'F_VERBA_ASS':F_VERBA_ASS, 
        'F_VERBA_VEN_REN':F_VERBA_VEN_REN, 
        'F_VERBA_VEN_DEC':F_VERBA_VEN_DEC, 
        'F_VERBA_VEN_SUP':F_VERBA_VEN_SUP, 
        'F_VERBA_VEN_TEC':F_VERBA_VEN_TEC,
        'F_VERBA_ASS_SEM':F_VERBA_ASS_SEM,
        'F_VERBA_VEN_REN_SEM':F_VERBA_VEN_REN_SEM, 
        'F_VERBA_VEN_DEC_SEM':F_VERBA_VEN_DEC_SEM, 
        'F_VERBA_VEN_SUP_SEM':F_VERBA_VEN_SUP_SEM, 
        'F_VERBA_VEN_TEC_SEM':F_VERBA_VEN_TEC_SEM}

In [314]:
# Nomes dos arquivos de verbas para RH
verbas_telha=nome_verbas(DIR_RH,dict_EMPRESAS[1])
verbas_tume=nome_verbas(DIR_RH,dict_EMPRESAS[4])

In [315]:
verbas_telha['F_VERBA_ASS_SEM']

'C:/apuracao/2022/mai/para_RH/VERBA_4118_SEGMENTO_ASSESSORES_TELHANORTE_sem_considerar_faltas_injustificadas.xlsx'

In [316]:
import shutil
# Fazendo backup das bases nas pasta ORIGINAIS respectiva ao mes e ano da apuração 
shutil.copy2(E_EXTRATO,F_EXTRATO)
shutil.copy2(E_FALTAS,F_FALTAS)

'C:/apuracao/2022/mai/ORIGINAIS/PREMIACOES TINA.xlsx'

In [317]:
# Função para importar comissões separamente por empresa (vendas da ADP)
def importa_comissoes(arquivo):
    # Atribuindo nomes às colunas das comissões
    column_names=[
    'N1',	
    'N2',	
    'N3',	
    'MATRICULA',	
    'POS_NEG',	 
    'VALOR',
    'NONE'
    ]
    print(arquivo)
    print("Importando as comissões")
    print("Arquivo: "+arquivo)
    comissao = pd.read_csv(arquivo, delimiter=";", names=column_names, header=None, skiprows=1)
    # Trocando "," por "." para poder calcular os números no Python
    comissao["VALOR"]=comissao["VALOR"].str.replace(",",".").astype(float)
    ##  Vendas devolvidas como negativas
    print("Valores negativos para vendas devolvidas")
    comissao["VALOR_COR"]=np.where(comissao["POS_NEG"]==3771, -comissao["VALOR"],comissao["VALOR"])
    # Só matrícula e valor das vendas correspondente a essa matrícula
    comissao=comissao[['MATRICULA','VALOR_COR']]
    # Somando as vendas totais de cada matrícula
    n_vendas = comissao.groupby(['MATRICULA']).size().reset_index()
    n_vendas.rename(columns={0:'NUMERO_DE_LOJAS_COM_VENDA'}, inplace=True)
    print("Somando todas as vendas de uma matrícula:")
    comissao = comissao.groupby(['MATRICULA'])['VALOR_COR'].sum().reset_index()
    comissao.rename(columns={'VALOR_COR':'VENDAS_TOT'}, inplace=True)
    comissao['COMISSAO']=comissao['VENDAS_TOT']*0.01
    comissao=pd.merge(comissao,n_vendas, how='left', on=['MATRICULA'] )
    return comissao

In [318]:
# Ricardo Valdir via e-mail para o Junior no dia da apuração
E_COMISSAO = DIR_ENTRA+"LANC_COMISSAO_"+str(I_MES)+ANO+'_'+dict_EMPRESAS[1].lower() # base de entrada Comissao
F_COMISSAO = DIR_IN+"LANC_COMISSAO_"+str(I_MES)+ANO+'_'+dict_EMPRESAS[1].lower() # cópia da base Comissão a ser mantida
# backup da base de comissão TELHANORTE
shutil.copy2(E_COMISSAO,F_COMISSAO)
# importando a comissão Telhanorte
comissao_telha=importa_comissoes(F_COMISSAO)


C:/apuracao/2022/mai/ORIGINAIS/LANC_COMISSAO_052022_telhanorte
Importando as comissões
Arquivo: C:/apuracao/2022/mai/ORIGINAIS/LANC_COMISSAO_052022_telhanorte
Valores negativos para vendas devolvidas
Somando todas as vendas de uma matrícula:


In [319]:
# Para visualizar as vendas totais e a comissão de um colaborador 
comissao_telha[comissao_telha['MATRICULA'] == 25724]

,MATRICULA,VENDAS_TOT,COMISSAO,NUMERO_DE_LOJAS_COM_VENDA
435,25724,79905.1,799.051,3


In [320]:
# Ricardo Valdir via e-mail para o Junior no dia da apuração
E_COMISSAO = DIR_ENTRA+"LANC_COMISSAO_"+str(I_MES)+ANO+'_'+dict_EMPRESAS[4].lower() # base de entrada Comissao
F_COMISSAO = DIR_IN+"LANC_COMISSAO_"+str(I_MES)+ANO+'_'+dict_EMPRESAS[4].lower() # cópia da base Comissão a ser mantida
# backup da base de comissão TUMELERO
shutil.copy2(E_COMISSAO,F_COMISSAO)
# importando a comissão Tumelero
comissao_tume=importa_comissoes(F_COMISSAO)


C:/apuracao/2022/mai/ORIGINAIS/LANC_COMISSAO_052022_tumelero
Importando as comissões
Arquivo: C:/apuracao/2022/mai/ORIGINAIS/LANC_COMISSAO_052022_tumelero
Valores negativos para vendas devolvidas
Somando todas as vendas de uma matrícula:


In [321]:
# Para visualizar as vendas totais e a comissão de um colaborador 
comissao_tume[comissao_tume['MATRICULA'] == 2359]

,MATRICULA,VENDAS_TOT,COMISSAO,NUMERO_DE_LOJAS_COM_VENDA
64,2359,44593.3,445.933,2


In [322]:
# juntando comissão da Telhanorte e Tumelero
comissao_telha['ID_UNN']=1 # Identidade da empresa Telhanorte = 1
comissao_tume['ID_UNN']=4 # Identidade da empresa Tumelero = 4
comissao=pd.concat([
    comissao_telha[['ID_UNN', 'MATRICULA', 'VENDAS_TOT', 'COMISSAO', 'NUMERO_DE_LOJAS_COM_VENDA']],
    comissao_tume[['ID_UNN', 'MATRICULA', 'VENDAS_TOT', 'COMISSAO', 'NUMERO_DE_LOJAS_COM_VENDA']]
    ])



In [323]:
# Lendo os alvos que os vendedores devem atingir por loja
print("Lendo alvos das lojas no arquivo:")
print(F_ALVO)
alvos = pd.read_csv(F_ALVO, delimiter=";", header=0, skiprows=0)
# Renomeando e transformando em número
alvos.rename(columns={'target_sales_salesperson':'Alvo1'}, inplace=True)
alvos['Alvo1']=alvos['Alvo1'].str.replace(",",".").astype(float)
#Transformando floor remuneration em Mínimo Garantido e em número
alvos['Minimo Garantido']=alvos['floor_remuneration'].str.replace(",",".").astype(float)

Lendo alvos das lojas no arquivo:
C:/apuracao/info/target_salesperson.csv


In [324]:
alvos.loc[[0,39]]

,id_unn,cod_loja,Alvo1,target_margin_salesperson_E1,target_margin_salesperson_E2,target_margin_salesperson_E3,target_sales_assistant,target_assistant_E1,target_assistant_E2,target_assistant_E3,floor_remuneration,base_remuneration,Minimo Garantido
0,1,75,160000.0,0,0,0,0,0,0,0,"1283,7",0,1283.7
39,4,510019,195000.0,0,0,0,0,0,0,0,"1653,0","1171,2",1653.0


In [325]:
# Mantendo somente alvos Telhanorte
#alvos=alvos[alvos['id_unn']==1] # id_unn = (1=Telhanorte; 2=Tumelero)
# Mantendo somente alguns parâmetros
alvos= alvos[['cod_loja','Alvo1', 'Minimo Garantido']]

alvos=pd.merge(alvos,nome_lojas[['cod_loja','sigla_loja']],how='left',on='cod_loja')

In [326]:
# Dicionário de alvo por codigo da loja
dict_alvo=dict(zip(alvos_telha['cod_loja'], alvos_telha['Alvo1']))

In [327]:
# Dicionário de alvo por codigo da loja
dict_minimo_garantido=dict(zip(alvos['cod_loja'], alvos_telha['Minimo Garantido']))

In [328]:
# Lendo os alvos que os vendedores devem atingir por loja
#print("Lendo alvos das lojas no arquivo:")
# print(F_ALVO)
# alvos = pd.read_excel(F_ALVO, header=0, skiprows=1)
# alvos['Minimo Garantido']=alvos['Minimo Garantido']/100.
#a[a['Loja']==7992]
# alvos.columns

In [329]:
# Lendo faltas injustificadas
print("Lendo faltas injustificadas no arquivo:")
print(F_FALTAS)
faltas_inj = pd.read_excel(F_FALTAS, header=0)

Lendo faltas injustificadas no arquivo:
C:/apuracao/2022/mai/ORIGINAIS/PREMIACOES TINA.xlsx


In [330]:
faltas = faltas_inj[(faltas_inj['DATA DA OCORRÊNCIA'] >= DATA_I) &
           (faltas_inj['DATA DA OCORRÊNCIA'] <= DATA_F)]
faltas['DATA DA OCORRÊNCIA'].sort_values()

9     2022-04-27
35    2022-04-27
151   2022-04-27
224   2022-04-27
244   2022-04-27
         ...    
75    2022-05-15
91    2022-05-15
99    2022-05-15
138   2022-05-15
74    2022-05-15
Name: DATA DA OCORRÊNCIA, Length: 196, dtype: datetime64[ns]

In [331]:
faltas=pd.merge(
    faltas.rename(columns={
        'EMPRESA':'ID_UNN',
        'NOME ESTAB':'SIGLA_LOJA'
        }),
    nome_lojas[['id_unn','cod_loja','sigla_loja']].rename(columns={
        'id_unn':'ID_UNN',
        'cod_loja':'COD_LOJA',
        'sigla_loja': 'SIGLA_LOJA'
        }),
    how='left',
    on=['ID_UNN', 'SIGLA_LOJA']
    )

In [332]:
faltas.head(5)

,PERIODO - DTINI,PERIODO - DTFIM,ID_UNN,DATA DA OCORRÊNCIA,COD ESTAB,SIGLA_LOJA,COD CR,NOME CR,CÓDIGO DO CENTRO CUSTO CONTABILIDADE,MATR CHEFIA,NOME CHEFIA,MATR EMPR,NOME EMPPR,FUNCAO-DENOMINACAO,COD VERBA,DESCRICAO VERBA,VALOR,COD_LOJA
0,2022-04-16,2022-05-15,1,2022-04-27,32,PAM,344402,PAM - OPERACOES LOJA,344402,11014,REGINALDO DIAS DO NASCIMENTO,20159,ADAILZA SANTOS DA SILVA,Lider de Caixa,9005,Falta Injustificada,7.33,34
1,2022-04-16,2022-05-15,1,2022-04-28,32,PAM,344402,PAM - OPERACOES LOJA,344402,11014,REGINALDO DIAS DO NASCIMENTO,20159,ADAILZA SANTOS DA SILVA,Lider de Caixa,9005,Falta Injustificada,7.33,34
2,2022-04-16,2022-05-15,1,2022-04-30,32,PAM,344402,PAM - OPERACOES LOJA,344402,11014,REGINALDO DIAS DO NASCIMENTO,20159,ADAILZA SANTOS DA SILVA,Lider de Caixa,9005,Falta Injustificada,7.33,34
3,2022-04-16,2022-05-15,1,2022-05-02,32,PAM,344402,PAM - OPERACOES LOJA,344402,11014,REGINALDO DIAS DO NASCIMENTO,20159,ADAILZA SANTOS DA SILVA,Lider de Caixa,9005,Falta Injustificada,7.33,34
4,2022-04-16,2022-05-15,1,2022-05-03,32,PAM,344402,PAM - OPERACOES LOJA,344402,11014,REGINALDO DIAS DO NASCIMENTO,20159,ADAILZA SANTOS DA SILVA,Lider de Caixa,9005,Falta Injustificada,7.33,34


In [333]:
# Loading Remuneration extract from Tina
print("Lendo Extrato de Remuneração da Tina")
print(F_EXTRATO)
extrato = pd.read_excel(F_EXTRATO)

Lendo Extrato de Remuneração da Tina
C:/apuracao/2022/mai/ORIGINAIS/extract_remuneration_20220527.xlsx


In [334]:
# Somente colaboradores Tina das lojas da safra
print("Considerando somente colaboradores das lojas participantes da Tina")
COLS_TO_KEEP=[
    'store_sk', 
    'registration_id', 
    'segment', 
    'job_simplified',
    'E1_gross_sales', 
    'E2_gross_sales', 
    'E3_gross_sales', 
    'gross_sales', 
    'sales_segment', 
    'rentability'
    ]
so_safra = extrato[extrato["store_sk"].isin(nome_lojas['cod_loja'])][COLS_TO_KEEP].copy(deep=True)
so_safra.columns

Considerando somente colaboradores das lojas participantes da Tina


Index(['store_sk', 'registration_id', 'segment', 'job_simplified',
       'E1_gross_sales', 'E2_gross_sales', 'E3_gross_sales', 'gross_sales',
       'sales_segment', 'rentability'],
      dtype='object')

In [335]:
faltas.columns

Index(['PERIODO - DTINI', 'PERIODO - DTFIM', 'ID_UNN', 'DATA DA OCORRÊNCIA',
       'COD ESTAB', 'SIGLA_LOJA', 'COD CR', 'NOME CR',
       'CÓDIGO DO CENTRO CUSTO CONTABILIDADE', 'MATR CHEFIA', 'NOME CHEFIA',
       'MATR EMPR', 'NOME EMPPR', 'FUNCAO-DENOMINACAO', 'COD VERBA',
       'DESCRICAO VERBA', 'VALOR', 'COD_LOJA'],
      dtype='object')

In [500]:
#para planilha checagem
tina=so_safra.copy(deep=True)


In [501]:
# Verificando quem tem falta injustificada
tina['FALTA']=np.where(
    (tina['store_sk'].isin(faltas['COD_LOJA'])==True) &
    (tina['registration_id'].isin(faltas['MATR EMPR'])==True) 
    ,'SIM','NAO')

In [502]:
tina.head()

,store_sk,registration_id,segment,job_simplified,E1_gross_sales,E2_gross_sales,E3_gross_sales,gross_sales,sales_segment,rentability,FALTA
0,24.0,6444.0,E3,salesperson,68415.974,4128.009,831848.878,904392.861,831848.878,96.759267,NAO
1,30.0,6313.0,E3,salesperson,17313.431,9263.325,680395.910,708643.896,680395.910,97.398870,NAO
2,23.0,8726.0,E1,salesperson,376880.689,3662.671,131237.714,511781.074,376880.689,97.977032,NAO
3,23.0,24935.0,E1,salesperson,284278.797,745.956,155565.081,440589.834,284278.797,104.334492,NAO
4,71.0,23522.0,E3,salesperson,57220.369,7249.369,484460.073,548929.811,484460.073,101.662767,NAO


In [503]:
# Juntando nome de sigla das lojas
tina=pd.merge(
    tina.rename(columns={'store_sk':'cod_loja'}),
    nome_lojas[['cod_loja', 'sigla_loja']],
    how='left',on='cod_loja'
    )

In [504]:
# Renomeando matricula para juntar com a comissão
tina.rename(columns={'registration_id':'MATRICULA'}, inplace=True)


In [505]:
# Identificando lojas da empresa (ID_UNN) Telhanorte = 1 e Tumelero = 4
tina['ID_UNN']=np.where(tina['cod_loja']<510000,1,4)

In [506]:
# # Juntando com a comissão
tina=tina.merge(
    comissao,
    how='left',on=['ID_UNN','MATRICULA'])

In [486]:
tina.sample(5)

,cod_loja,MATRICULA,segment,job_simplified,E1_gross_sales,E2_gross_sales,E3_gross_sales,gross_sales,sales_segment,rentability,FALTA,sigla_loja,ID_UNN,VENDAS_TOT,COMISSAO,NUMERO_DE_LOJAS_COM_VENDA
318,64.0,24488.0,E1,salesperson,56385.520,17120.096,118957.184,192462.800,56385.520,99.712887,NAO,SAN,1,192462.79,1924.6279,1.0
401,73.0,24329.0,E1,salesperson,24453.856,3955.000,73521.455,101930.311,24453.856,103.687501,NAO,TBT,1,101930.31,1019.3031,1.0
555,3.0,29293.0,E1,-,-28.500,0.000,291.600,263.100,-28.500,88.564808,SIM,MGN,1,263.10,2.6310,2.0
745,12.0,27015.0,E3,salesperson,23651.824,10226.471,71488.825,105367.120,71488.825,102.026065,NAO,JDI,1,105367.12,1053.6712,1.0
360,32.0,26716.0,E3,salesperson,45593.117,9453.293,97994.193,153040.603,97994.193,97.298077,NAO,GRU,1,153040.60,1530.4060,1.0


In [507]:

tina=pd.merge(
    tina,
    alvos[['cod_loja','Alvo1', 'Minimo Garantido']],
    how='left', on='cod_loja'
    )

In [488]:
tina.sample(5)

,cod_loja,MATRICULA,segment,job_simplified,E1_gross_sales,E2_gross_sales,E3_gross_sales,gross_sales,sales_segment,rentability,FALTA,sigla_loja,ID_UNN,VENDAS_TOT,COMISSAO,NUMERO_DE_LOJAS_COM_VENDA,Alvo1,Minimo Garantido
599,19.0,998.0,NaN,-,-9150.848,-8770.920,-20133.342,-38055.110,-20133.342,107.086535,NAO,IMI,1,-38055.11,-380.5511,1.0,270000.0,1985.0
987,75.0,28511.0,E2,assistant,0.000,0.000,0.000,0.000,0.000,0.000000,NAO,AMZ,1,NaN,NaN,NaN,160000.0,1283.7
530,24.0,13216.0,E3,salesperson,15692.374,12681.234,152462.218,180835.826,152462.218,101.841899,NAO,ARI,1,180835.83,1808.3583,1.0,270000.0,1985.0
294,64.0,24839.0,E3,salesperson,35880.070,11329.981,92312.204,139522.255,92312.204,102.764241,NAO,SAN,1,139522.25,1395.2225,1.0,256003.2,2133.4
301,64.0,6790.0,E3,salesperson,18239.279,8072.279,149799.867,176111.425,149799.867,102.327503,NAO,SAN,1,176111.42,1761.1142,1.0,256003.2,2133.4


In [508]:
# Eliminando alvos de assessores
#tina['Alvo1']=np.where(tina['job_simplified']=='assistant',np.nan,tina['Alvo1'])
#tina['VENDAS_SOBRE_ALVO']=tina['VENDAS_TOT']/tina['Alvo1']

tina.loc[tina.job_simplified=='assistant', 'Alvo1'] = None
tina['VENDAS_SOBRE_ALVO'] =  None
tina.loc[tina.job_simplified=='salesperson', 'VENDAS_SOBRE_ALVO'] = tina['VENDAS_TOT']/tina['Alvo1']
tina.sample()
#tina.loc[
#    (tina.ID_UNN == 1) &  # ID_UNN para Telhanorte
#    (tina.job_simplified=='salesperson'), 
#    'PREMIO_RENTABILIDADE'
#    ] = tina.apply(
#        lambda x: alvo_bonus_rentabilidade(
#            1,  # ID_UNN para Telhanorte
#            x['VENDAS_TOT'], x['rentability'], x['sales_segment'], x['Alvo1']
#            ), axis=1)

,cod_loja,MATRICULA,segment,job_simplified,E1_gross_sales,E2_gross_sales,E3_gross_sales,gross_sales,sales_segment,rentability,FALTA,sigla_loja,ID_UNN,VENDAS_TOT,COMISSAO,NUMERO_DE_LOJAS_COM_VENDA,Alvo1,Minimo Garantido,VENDAS_SOBRE_ALVO
395,25.0,17702.0,E3,salesperson,41690.599,11844.059,145098.526,199291.184,145098.526,103.002006,NAO,ZNO,1,199291.18,1992.9118,1.0,238200.0,1985.0,0.836655


In [497]:
tina[tina['job_simplified']=='assistant']

,cod_loja,MATRICULA,segment,job_simplified,E1_gross_sales,E2_gross_sales,E3_gross_sales,gross_sales,sales_segment,rentability,FALTA,sigla_loja,ID_UNN,VENDAS_TOT,COMISSAO,NUMERO_DE_LOJAS_COM_VENDA,Alvo1,Minimo Garantido,VENDAS_SOBRE_ALVO,ATINGIU_ALVO
61,70.0,29013.0,E1,assistant,81661.088,22664.598,151183.668,255509.354,81661.088,97.466854,NAO,WHL,1,255509.34,2555.0934,1.0,NaN,NaN,None,50mil
118,70.0,25780.0,E3,assistant,50377.266,21778.256,100190.470,172345.992,100190.470,100.162187,NAO,WHL,1,172345.99,1723.4599,1.0,NaN,NaN,None,50mil
126,9.0,24914.0,E1,assistant,54222.412,12486.591,117569.373,184278.376,54222.412,100.162116,NAO,SA1,1,184278.37,1842.7837,1.0,NaN,NaN,None,50mil
128,70.0,28316.0,E3,assistant,49176.172,13812.881,110765.437,173754.490,110765.437,100.810001,NAO,WHL,1,176921.50,1769.2150,2.0,NaN,NaN,None,50mil
147,70.0,28501.0,E2,assistant,73008.092,15165.938,80358.152,168532.182,15165.938,85.629219,NAO,WHL,1,168532.18,1685.3218,1.0,NaN,NaN,None,NAO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024,71.0,27566.0,E1,assistant,2532.041,2376.910,46289.971,51198.922,2532.041,107.998580,NAO,RDV,1,53356.09,533.5609,1.0,NaN,NaN,None,NAO
1025,74.0,27581.0,E2,assistant,7120.231,8492.799,41036.010,56649.040,8492.799,91.731068,NAO,RPT,1,56649.04,566.4904,1.0,NaN,NaN,None,NAO
1026,71.0,28261.0,E2,assistant,8774.976,2157.700,56567.964,67500.640,2157.700,85.050257,NAO,RDV,1,67506.14,675.0614,1.0,NaN,NaN,None,NAO
1027,39.0,28612.0,E1,assistant,22249.440,2998.659,29336.112,54584.211,22249.440,95.860376,NAO,UBE,1,54584.21,545.8421,1.0,NaN,NaN,None,NAO


In [509]:
# Eliminando minimos garantidos de assessores
tina['Minimo Garantido']=np.where(tina['job_simplified']=='assistant',np.nan,tina['Minimo Garantido'])

In [494]:
# TODO DELETAR SE NÃO DER ERRO
# Eliminando vendas sobre alvo de assessores
# tina['VENDAS_SOBRE_ALVO']=np.where(tina['job_simplified']=='assistant',np.nan,tina['VENDAS_SOBRE_ALVO'])

In [511]:
# Vendedores atingiu alvo ou não atingiu alvo
tina.loc[
    (tina.job_simplified=='salesperson') &
    (tina.VENDAS_SOBRE_ALVO>=1.), 'ATINGIU_ALVO'] = 'SIM'
tina.loc[
    (tina.job_simplified=='salesperson') &
    (tina.VENDAS_SOBRE_ALVO<1.), 'ATINGIU_ALVO'] = 'NAO'

# TODO DELETAR LINHA SE 
# tina['ATINGIU_ALVO']=np.where(tina['VENDAS_SOBRE_ALVO']>=1.,'SIM','NAO')

In [512]:
# Assessores atingiu alvo
tina.loc[
    (tina.job_simplified=='assistant') &
    (tina.sales_segment>=30000), 'ATINGIU_ALVO'] = '30mil'
tina.loc[
    (tina.job_simplified=='assistant') &
    (tina.sales_segment>=50000), 'ATINGIU_ALVO'] = '50mil'
tina.loc[
    (tina.job_simplified=='assistant') &
    (tina.sales_segment<30000), 'ATINGIU_ALVO'] = 'NAO'


In [516]:
# RECEBE PREMIAÇÃO TINA SE ATINGIU ALVO E NÃO TEM FALTA INJUSTIFICADA
tina['RECEBE_PREMIACAO']=np.where((
    (tina['ATINGIU_ALVO']=='SIM') |
    (tina['ATINGIU_ALVO']=='50mil') |
    (tina['ATINGIU_ALVO']=='30mil')) & 
    (tina['FALTA']=='NAO'),
     'SIM', 'NAO')


In [521]:
#tina[['MATRICULA','VENDAS_TOT', 'ATINGIU_ALVO','RECEBE_PREMIACAO']][tina['RECEBE_PREMIACAO']!='NAO']
#tina[['MATRICULA','VENDAS_TOT', 'FALTA', 'ATINGIU_ALVO','RECEBE_PREMIACAO']][tina['ATINGIU_ALVO']!='NAO'].sample(15)
tina[['MATRICULA','VENDAS_TOT', 'FALTA', 'ATINGIU_ALVO','RECEBE_PREMIACAO']][tina['FALTA']=='SIM'].sample(15)

,MATRICULA,VENDAS_TOT,FALTA,ATINGIU_ALVO,RECEBE_PREMIACAO
568,131.0,80448.28,SIM,NAO,NAO
489,28997.0,887.02,SIM,NAO,NAO
778,28427.0,39440.86,SIM,NAO,NAO
902,23612.0,158182.45,SIM,NAO,NAO
551,20322.0,61751.25,SIM,NAO,NAO
932,28233.0,45694.22,SIM,NAO,NAO
110,28478.0,206066.79,SIM,SIM,NAO
517,28870.0,-119.80,SIM,NAO,NAO
538,27877.0,113260.32,SIM,NAO,NAO
555,29293.0,263.10,SIM,NaN,NAO


In [522]:
#Salvando verba para TELHANORTE ou TUMELERO
# Entradas: v verba, n nome_verba, x numero_verba, f f_nome
def salva_verba(empresa,verba,numero_verba,arquivo):
#    v.rename(columns={ v.columns[0]: 'MATRICULA'}, inplace=True)
    verba.rename(columns={ verba.columns[1]: "VALOR" }, inplace=True)
    verba['ZERO'] = 0
    verba['VERBA'] = numero_verba
    if empresa == 'TELHANORTE':
        verba=verba[['MATRICULA', 'ZERO', 'VERBA','VALOR']]
    elif empresa == 'TUMELERO':
         verba=verba[['MATRICULA', 'VERBA', 'ZERO','VALOR']]
    else: 
        print('Erro!!! Não veio a empresa nem como TELHANORTE e nem como TUMELERO')
        return
    verba = verba[verba['VALOR'] > 0]
    # Saving as a excel file for RH
    verba.to_excel(arquivo, sheet_name="VERBA "+str(numero_verba), index=False)
    

In [471]:
tina[tina['VENDAS_SOBRE_ALVO']>1.]

,ID_UNN,cod_loja,sigla_loja,MATRICULA,segment,job_simplified,E1_gross_sales,E2_gross_sales,E3_gross_sales,gross_sales,sales_segment,rentability,VENDAS_TOT,COMISSAO,NUMERO_DE_LOJAS_COM_VENDA,Alvo1,Minimo Garantido,FALTA,VENDAS_SOBRE_ALVO,ATINGIU_ALVO,RECEBE_PREMIACAO,PREMIO_RENTABILIDADE
0,1,24.0,ARI,6444.0,E3,salesperson,68415.974,4128.009,831848.878,904392.861,831848.878,96.759267,904392.86,9043.9286,1.0,270000.0,1985.0,NAO,3.349603,SIM,NAO,0.000000
1,1,30.0,MOR,6313.0,E3,salesperson,17313.431,9263.325,680395.910,708643.896,680395.910,97.398870,708643.90,7086.4390,1.0,270000.0,1985.0,NAO,2.624607,SIM,NAO,0.000000
2,1,23.0,PMG,8726.0,E1,salesperson,376880.689,3662.671,131237.714,511781.074,376880.689,97.977032,511781.08,5117.8108,1.0,238200.0,1985.0,NAO,2.148535,SIM,NAO,0.000000
3,1,23.0,PMG,24935.0,E1,salesperson,284278.797,745.956,155565.081,440589.834,284278.797,104.334492,440589.84,4405.8984,1.0,238200.0,1985.0,NAO,1.849663,SIM,NAO,1432.204169
4,1,71.0,RDV,23522.0,E3,salesperson,57220.369,7249.369,484460.073,548929.811,484460.073,101.662767,548929.81,5489.2981,1.0,245148.0,2042.9,NAO,2.239177,SIM,NAO,1005.544172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,1,9.0,SA1,27390.0,E3,salesperson,22069.452,4062.498,223820.870,249952.820,223820.870,98.453848,249952.81,2499.5281,1.0,214875.6,1983.7,NAO,1.163244,SIM,NAO,0.000000
194,1,27.0,OSA,28873.0,E3,salesperson,30050.810,7708.054,198299.371,236058.235,198299.371,95.133208,236058.23,2360.5823,1.0,202824.0,1972.0,NAO,1.163857,SIM,NAO,0.000000
195,1,38.0,DPE,11880.0,E1,salesperson,25884.578,15563.124,159194.087,200641.789,25884.578,100.507805,200641.79,2006.4179,1.0,198000.0,1929.2,NAO,1.013342,SIM,NAO,213.144331
198,1,37.0,PIR,27521.0,E3,salesperson,44385.744,8507.263,159462.440,212355.447,159462.440,94.814847,212355.45,2123.5545,1.0,205000.0,1949.0,NAO,1.035880,SIM,NAO,0.000000


In [456]:
tina.columns

Index(['cod_loja', 'MATRICULA', 'segment', 'job_simplified', 'E1_gross_sales',
       'E2_gross_sales', 'E3_gross_sales', 'gross_sales', 'sales_segment',
       'rentability', 'FALTA', 'sigla_loja', 'ID_UNN', 'VENDAS_TOT',
       'COMISSAO', 'NUMERO_DE_LOJAS_COM_VENDA', 'Alvo1', 'Minimo Garantido',
       'VENDAS_SOBRE_ALVO', 'ATINGIU_ALVO', 'RECEBE_PREMIACAO'],
      dtype='object')

In [523]:
# Reorganizando a ordem dos parâmtros
tina = tina[['ID_UNN', 'cod_loja', 'sigla_loja', 'MATRICULA', 'segment', 'job_simplified', 'E1_gross_sales',
       'E2_gross_sales', 'E3_gross_sales', 'gross_sales', 'sales_segment',
       'rentability', 'VENDAS_TOT',
       'COMISSAO', 'NUMERO_DE_LOJAS_COM_VENDA', 'Alvo1', 'Minimo Garantido',
       'FALTA', 'VENDAS_SOBRE_ALVO', 'ATINGIU_ALVO', 'RECEBE_PREMIACAO']]

In [524]:
def alvo_bonus_rentabilidade(id_unn, vendas_tot,rentabilidade,vendas_seg,alvo):
#     # Modelo Vendedores Prêmio Rentabilidade
#    T E S T A D O     E     C H E C A D O
# gat_min_rentab = 100.  # Tem que bater 100% da rentabilidade
# ang_coef_rentab = 0.001  # Coeficiente angular da rentabilidade = 0.1%
# lin_coef_rentab = -0.1  # Coeficiente linear da rentabilidade = -0.1
# kicker = 200.
    global GAT_MIN_RENTAB, ANG_COEF_RENTAB, LIN_COEF_RENTAB, KICKER
    if vendas_tot>=alvo and rentabilidade >= GAT_MIN_RENTAB:
    #    # TODO DELETAR SE OK
#        val_rent = KICKER DELETAR SE OK
        # TODO DELETAR SE OK
        #if (rentabilidade-GAT_MIN_RENTAB)>0: DELETAR SE OK
        val_rent=(rentabilidade*ANG_COEF_RENTAB+LIN_COEF_RENTAB)*vendas_seg+KICKER[id_unn]
            # val_rent=val_rent+int(r-gat_min_rentab)*ang_coef_rentab*s
    else:
        val_rent=0.0
    return val_rent



In [526]:
# Calculando Prêmio Rentabilidade para TELHANORTE
tina.loc[
    (tina.ID_UNN == 1) &  # ID_UNN para Telhanorte
    (tina.job_simplified=='salesperson'), 
    'PREMIO_RENTABILIDADE'
    ] = tina.apply(
        lambda x: alvo_bonus_rentabilidade(
            1,  # ID_UNN para Telhanorte
            x['VENDAS_TOT'], x['rentability'], x['sales_segment'], x['Alvo1']
            ), axis=1).copy(deep=True)

In [460]:
# Calculando Prêmio Rentabilidade para TUMELERO
tina.loc[
    (tina.ID_UNN == 4) &  # ID_UNN para Tumelero
    (tina.job_simplified=='salesperson'), 
    'PREMIO_RENTABILIDADE'
    ] = tina.apply(
        lambda x: alvo_bonus_rentabilidade(
            4, # ID_UNN para Tumelero
            x['VENDAS_TOT'], x['rentability'], x['sales_segment'], x['Alvo1']
            ), axis=1).copy(deep=True)

In [548]:
# CHECANDO
tina[(tina['ID_UNN']==1) &
#(tina['job_simplified']=='salesperson') &
(tina['RECEBE_PREMIACAO']=='SIM')]

,ID_UNN,cod_loja,sigla_loja,MATRICULA,segment,job_simplified,E1_gross_sales,E2_gross_sales,E3_gross_sales,gross_sales,sales_segment,rentability,VENDAS_TOT,COMISSAO,NUMERO_DE_LOJAS_COM_VENDA,Alvo1,Minimo Garantido,FALTA,VENDAS_SOBRE_ALVO,ATINGIU_ALVO,RECEBE_PREMIACAO,PREMIO_RENTABILIDADE,PREMIO_DECORACAO
0,1,24.0,ARI,6444.0,E3,salesperson,68415.974,4128.009,831848.878,904392.861,831848.878,96.759267,904392.86,9043.9286,1.0,270000.0,1985.0,NAO,3.349603,SIM,SIM,0.000000,82.56018
1,1,30.0,MOR,6313.0,E3,salesperson,17313.431,9263.325,680395.910,708643.896,680395.910,97.398870,708643.90,7086.4390,1.0,270000.0,1985.0,NAO,2.624607,SIM,SIM,0.000000,185.26650
2,1,23.0,PMG,8726.0,E1,salesperson,376880.689,3662.671,131237.714,511781.074,376880.689,97.977032,511781.08,5117.8108,1.0,238200.0,1985.0,NAO,2.148535,SIM,SIM,0.000000,73.25342
3,1,23.0,PMG,24935.0,E1,salesperson,284278.797,745.956,155565.081,440589.834,284278.797,104.334492,440589.84,4405.8984,1.0,238200.0,1985.0,NAO,1.849663,SIM,SIM,1432.204169,14.91912
4,1,71.0,RDV,23522.0,E3,salesperson,57220.369,7249.369,484460.073,548929.811,484460.073,101.662767,548929.81,5489.2981,1.0,245148.0,2042.9,NAO,2.239177,SIM,SIM,1005.544172,144.98738
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
937,1,38.0,DPE,29505.0,NaN,assistant,20468.352,12576.714,41564.850,74609.916,41564.850,95.956476,75997.05,759.9705,1.0,NaN,NaN,NAO,None,30mil,SIM,NaN,NaN
948,1,38.0,DPE,29769.0,NaN,assistant,8204.877,11936.037,33409.703,53550.617,33409.703,101.276595,53550.62,535.5062,1.0,NaN,NaN,NAO,None,30mil,SIM,NaN,NaN
1012,1,29.0,PAR,29660.0,E1,assistant,34135.569,220.044,19346.848,53702.461,34135.569,101.995742,53956.24,539.5624,1.0,NaN,NaN,NAO,None,30mil,SIM,NaN,NaN
1015,1,72.0,SCA,24854.0,E3,assistant,19423.323,12152.512,31459.598,63035.433,31459.598,98.664527,63035.43,630.3543,1.0,NaN,NaN,NAO,None,30mil,SIM,NaN,NaN


In [528]:
# Salva verba rentabilidade TELHANORTE -->  salva_verba(empresa,verba,numero_verba,arquivo)
salva_verba(
    dict_EMPRESAS[1], 
    tina[['MATRICULA','PREMIO_RENTABILIDADE']][tina['ID_UNN']==1].copy(deep=True),
    4119,
    verbas_telha['F_VERBA_VEN_REN_SEM']) # verbas_telha['F_VERBA_ASS_SEM']
salva_verba(
    dict_EMPRESAS[1], 
    tina[['MATRICULA','PREMIO_RENTABILIDADE']][
        (tina['ID_UNN']==1) &
        (tina['RECEBE_PREMIACAO']=='SIM')].copy(deep=True),
        4119,
        verbas_telha['F_VERBA_VEN_REN'])


In [532]:
# Salva verba rentabilidade TUMELERO -->  salva_verba(empresa,verba,numero_verba,arquivo)
salva_verba(
    dict_EMPRESAS[4], # ID_UNN TUMELERO
    tina[['MATRICULA','PREMIO_RENTABILIDADE']][tina['ID_UNN']==4].copy(deep=True),
    4119,
    verbas_tume['F_VERBA_VEN_REN_SEM']) # verbas_tume['F_VERBA_ASS_SEM']
salva_verba(
    dict_EMPRESAS[4],  # ID_UNN TUMELERO
    tina[['MATRICULA','PREMIO_RENTABILIDADE']][
        (tina['ID_UNN']==4) &  # ID_UNN TUMELERO
        (tina['RECEBE_PREMIACAO']=='SIM')].copy(deep=True),
        4119,
        verbas_tume['F_VERBA_VEN_REN']) # verbas_tume['F_VERBA_VEN_REN']


In [529]:
# Calculating bonus for SALESPERSON TELHANORTE E TUMELERO
# NÃO TEM DIFERENÇA ENTRE EMPRESAS
# Verba 4120 - Segmento vendedores - DECORAÇÃO
def alvo_bonus_decoracao(vendas_seg,vendas_tot,alvo):
# Modelo Vendedores Prêmio DECORAÇÃO vendas
#    T E S T A D O     E     C H E C A D O
# Modelo Vendedores Comissão Turbinada Decoração
# Como as safras 1, 2 e 3 tem os mesmos valores, mantive só um valor para todas as safras
# gat_min_decoracao = 0.0  # Gatilho minimo decoração
# ang_coef_decoracao = 0.02  # Coeficiente angular decoração = 2%
# lin_coef_decoracao = 0.0  # Coeficiente Linear decoração = 0.0%

    global ANG_COEF_DECORACAO 
    if vendas_tot>=alvo:
        bonus = vendas_seg*ANG_COEF_DECORACAO
    else:
        bonus=0.0
    return bonus


In [534]:
# Calculando Verba DECORAÇÃO para TELHANORTE E TUMELERO
# SEM DIFERENÇA ENTRE EMPRESAS
tina.loc[
    tina.job_simplified=='salesperson', 
    'PREMIO_DECORACAO'
    ] = tina.apply(
        lambda x: alvo_bonus_decoracao(
            x['E2_gross_sales'], x['VENDAS_TOT'], x['Alvo1']
            ), axis=1).copy(deep=True)


In [536]:
# Salva verba decoracao TELHANORTE -->  salva_verba(empresa,verba,numero_verba,arquivo)
salva_verba(
    dict_EMPRESAS[1], 
    tina[['MATRICULA','PREMIO_DECORACAO']][tina['ID_UNN']==1].copy(deep=True),
    4120,
    verbas_telha['F_VERBA_VEN_DEC_SEM']) # verbas_telha
salva_verba(
    dict_EMPRESAS[1], 
    tina[['MATRICULA','PREMIO_DECORACAO']][
        (tina['ID_UNN']==1) &
        (tina['RECEBE_PREMIACAO']=='SIM')].copy(deep=True),
        4120,
        verbas_telha['F_VERBA_VEN_DEC']) # verbas_telha

In [547]:
# Calculating bonus for SALESPERSON considering differences for TELHANORTE e TUMELERO 
def alvo_bonus_superacao(id_unn, vendas_tot): # id_unn 1=Telhanorte e 4=Tumelero
# Modelo Vendedores Prêmio superação vendas
# T E S T A D O    E    C H E C A D O 
#M1 = 100  # meta1 = 100%
#M2 = 150  # meta2 = 150%
#M3 = 200  # meta3 = 200%
#M4 = 250  # meta4 = 250%
#B1 = {1:300, 4:200}  # bonus1 -> ganha se bater a meta1; Telhanorte 1 = R$300,00; Tumelero 4= R$200,00
#B2 = {1:600, 4:400}  # bonus2 -> ganha se bater a meta2; Telhanorte 1 = R$600,00; Tumelero 4= R$400,00
#B3 = {1:900, 4:600}  # bonus3 -> ganha se bater a meta3; Telhanorte 1 = R$900,00; Tumelero 4= R$600,00
#B4 = {1:1200, 4:800}  # bonus4  -> ganha se bater a meta4; Telhanorte 1 = R$1200,00; Tumelero 4= R$800,00
    global M1, M2, M3, M4, B1, B2, B3, B4 # metas e respectivos bonus
    if vendas_tot>=M4:
        bonus = B4[id_unn]
    elif vendas_tot>=M3:
        bonus = B3[id_unn]
    elif vendas_tot>=M2:
        bonus= B2[id_unn]
    elif vendas_tot>=M1:
        bonus=B1[id_unn]
    else:
        bonus=0.0
    return bonus



In [554]:
# Calculando Prêmio Superação para TELHANORTE
tina.loc[
    (tina.ID_UNN == 1) &  # ID_UNN para TELHANORTE
    (tina.job_simplified=='salesperson'), 
   'PREMIO_SUPERACAO'
    ] = tina.apply(
        lambda x: alvo_bonus_superacao(
            1, # ID_UNN para TELHANORTE
            x['VENDAS_TOT']/x['Alvo1']*100.
            ), axis=1).copy(deep=True)


In [555]:
# Calculando Prêmio Superação para TUMELERO
tina.loc[
    (tina.ID_UNN == 4) &  # ID_UNN para Tumelero
    (tina.job_simplified=='salesperson'), 
   'PREMIO_SUPERACAO'
    ] = tina.apply(
        lambda x: alvo_bonus_superacao(
            4, # ID_UNN para Tumelero
            x['VENDAS_TOT']/x['Alvo1']*100.
            ), axis=1).copy(deep=True)

In [558]:
# Salva verba superação TELHANORTE -->  salva_verba(empresa,verba,numero_verba,arquivo)
salva_verba(
    dict_EMPRESAS[1], #
    tina[['MATRICULA','PREMIO_SUPERACAO']][tina['ID_UNN']==1].copy(deep=True),
    4121,
    verbas_telha['F_VERBA_VEN_SUP_SEM']) # verbas_telha['F_VERBA_ASS_SEM']
salva_verba(
    dict_EMPRESAS[1], 
    tina[['MATRICULA','PREMIO_SUPERACAO']][
        (tina['ID_UNN']==1) &
        (tina['RECEBE_PREMIACAO']=='SIM')].copy(deep=True),
        4121,
        verbas_telha['F_VERBA_VEN_SUP'])


In [559]:
# Salva verba superação TUMELERO -->  salva_verba(empresa,verba,numero_verba,arquivo)
salva_verba(
    dict_EMPRESAS[4], # TUMELERO 
    tina[['MATRICULA','PREMIO_SUPERACAO']][tina['ID_UNN']==4].copy(deep=True),
    4121,
    verbas_tume['F_VERBA_VEN_SUP_SEM']) # verbas TUMELERO['F_VERBA_ASS_SEM']
salva_verba(
    dict_EMPRESAS[4], # TUMELERO 
    tina[['MATRICULA','PREMIO_SUPERACAO']][
        (tina['ID_UNN']==4) & # TUMELERO 
        (tina['RECEBE_PREMIACAO']=='SIM')].copy(deep=True),
        4121,
        verbas_tume['F_VERBA_VEN_SUP']) # verbas TUMELERO


In [565]:
# para TELHANORTE E TUMELERO SEM DIFERENÇA ENTRE EMPRESAS
def alvo_bonus_areastecnicas(vendas_seg,vendas_tot,alvo):
# Modelo Vendedores Prêmio ÁREAS TÉCNICAS
#    T E S T A D O     E     C H E C A D O
# Modelo Vendedores Comissão  ÁREAS TÉCNICAS
# Como as safras 1, 2 e 3 tem os mesmos valores, mantive só um valor para todas as safras
# gat_min_areastecnicas = 0.0  # Gatilho minimo Áreas Técnicas = 0.0%
# ang_coef_areastecnicas = 0.005  # Coeficiente angular Áreas Técnicas = 0.5%
# lin_coef_areastecnicas = 0.0  # Coeficiente Linear decoração = 0.0%

    global ANG_COEF_AREASTECNICAS # gat_min_areastecnicas, ang_coef_areastecnicas, lin_coef_areastecnicas 
    if vendas_tot>=alvo:
        bonus = vendas_seg*ANG_COEF_AREASTECNICAS
    else:
        bonus=0.0
    return bonus




In [566]:
# Calculando Verba DECORAÇÃO para TELHANORTE E TUMELERO
# SEM DIFERENÇA ENTRE EMPRESAS
#tina.loc[
#    tina.job_simplified=='salesperson', 
#    'PREMIO_DECORACAO'
#    ] = tina.apply(
#        lambda x: alvo_bonus_decoracao(
#            x['E2_gross_sales'], x['VENDAS_TOT'], x['Alvo1']
#            ), axis=1).copy(deep=True)

tina.loc[
    tina.job_simplified=='salesperson', 
    'PREMIO_AREAS_TECNICAS'
    ] = tina.apply(
        lambda x: alvo_bonus_areastecnicas(
            x['E1_gross_sales'],x['VENDAS_TOT'],x['Alvo1']
            ), axis=1).copy(deep=True)



# TODO DELETAR SE OK
# Calculando Premiação Individual ÁREAS TÉCNICAS
#tina['AREAS_TEC'] = np.where(
#    tina['job_simplified']=='salesperson',
 #   tina.apply(
#    lambda x: alvo_bonus_areastecnicas(x['E1_gross_sales'],x['VENDAS_TOT'],x['Alvo1']), axis=1).copy(),np.nan
#    )

In [569]:
# Salva verba áreas técnicas TELHANORTE -->  salva_verba(empresa,verba,numero_verba,arquivo)
salva_verba(
    dict_EMPRESAS[1], # TELHANORTE
    tina[['MATRICULA','PREMIO_AREAS_TECNICAS']][tina['ID_UNN']==1].copy(deep=True), # TELHANORTE
    4122,
    verbas_telha['F_VERBA_VEN_TEC_SEM']) # verbas # TELHANORTE
salva_verba(
    dict_EMPRESAS[1], # TELHANORTE
    tina[['MATRICULA','PREMIO_AREAS_TECNICAS']][
        (tina['ID_UNN']==1) & # TELHANORTE
        (tina['RECEBE_PREMIACAO']=='SIM')].copy(deep=True),
        4122,
        verbas_telha['F_VERBA_VEN_TEC']) # verbas # TELHANORTE


In [570]:
# Salva verba áreas técnicas TUMELERO -->  salva_verba(empresa,verba,numero_verba,arquivo)
salva_verba(
    dict_EMPRESAS[4], # TUMELERO=4
    tina[['MATRICULA','PREMIO_AREAS_TECNICAS']][tina['ID_UNN']==4].copy(deep=True), # TUMELERO=4
    4122,
    verbas_tume['F_VERBA_VEN_TEC_SEM']) # verbas # TUMELERO
salva_verba(
    dict_EMPRESAS[4], # TUMELERO=4
    tina[['MATRICULA','PREMIO_AREAS_TECNICAS']][
        (tina['ID_UNN']==4) & # TUMELERO=4
        (tina['RECEBE_PREMIACAO']=='SIM')].copy(deep=True),
        4122,
        verbas_tume['F_VERBA_VEN_TEC']) # verbas TUMELERO


In [573]:
# Calculando Prêmio Assessor SOMENTE TELHANORTE tem assessores
def alvo_bonus_assessor(vendas_seg):
    global ASS_A1, ASS_A2, ASS_B1, ASS_B2 # ass_a1, ass_a2, ass_b1, ass_b2
    if vendas_seg >= ASS_A2:
        bonus = ASS_B2
    elif vendas_seg >= ASS_A1:
        bonus = ASS_B1
    else:
        bonus = 0.0
    return bonus
    

In [574]:
# Calculando prêmio dos assessores - somente TELHANORTE tem assessores
tina['PREMIO_ASSESSOR'] = np.where(
    tina['job_simplified']=='assistant',
    tina.apply(
    lambda x: alvo_bonus_assessor(x['sales_segment']), axis=1).copy(),np.nan
    )

    

In [577]:
# Salva verba assessores somente TELHANORTE -->  salva_verba(empresa,verba,numero_verba,arquivo)
salva_verba(
    dict_EMPRESAS[1], # TELHANORTE
    tina[['MATRICULA','PREMIO_ASSESSOR']][tina['ID_UNN']==1].copy(deep=True), # TELHANORTE
    4118, #VERBA PREMIAÇÃO ASSESSOR
    verbas_telha['F_VERBA_ASS_SEM']) # verbas # TELHANORTE
salva_verba(
    dict_EMPRESAS[1], # TELHANORTE
    tina[['MATRICULA','PREMIO_ASSESSOR']][
        (tina['ID_UNN']==1) & # TELHANORTE
        (tina['RECEBE_PREMIACAO']=='SIM')].copy(deep=True),
        4118, #VERBA PREMIAÇÃO ASSESSOR
        verbas_telha['F_VERBA_ASS']) # verbas # TELHANORTE


In [578]:
tina['DIFERENCA_VENDAS_ADP_totTINAsegm']=tina['VENDAS_TOT']-tina['gross_sales']

In [ ]:
tina[tina['MATRICULA']==3343]

In [ ]:
tina.columns

In [579]:
tina.to_excel(F_SAI_CHECAGEM)